# Inscopix python API - imaging processing
### This notebook contains the procedure from preprocessing to ROI detection by CNMFe

### Import packages 
the inscopix python API is located in the same folder of data processing software. The isx.cnmfe is used for ROI detection.

### Remember to set up a ipcluster with following script under the isxenv
ipcluster start -n 2

In [1]:
import sys
sys.path.append("/Program Files/Inscopix/Data Processing/")
import os
import glob
import isx
# import isx.cnmfe

from datetime import datetime
start_time = datetime.now()
print('Start time: {}'.format(start_time))

Start time: 2021-08-11 14:52:27.901325


### File path and recording names
Deinfe path and the recording names of our recordings.<br>

In [3]:
path_to_file = 'F:/inscopix_nVista3/'
# path_to_file = 'E:/Inscopix_2021_nVista3/'
data_dir = os.path.join(path_to_file, 'BES0417210105') # Only change here
file_names_in_path = []
file_names_in_path = sorted(set(os.listdir(data_dir)))

# rec_name = '2020-09-02-15-58-16_video'
rec_name = file_names_in_path[1].split('.')[0]#.split('-')[0]
# rec_name = file_names_in_path[8][:-8] # This should be the first/sec raw file in the list with the time stamp
series_rec_names = {
        'day_0':
        [
            rec_name,
        ]
}
print(data_dir)
print(series_rec_names)

F:/inscopix_nVista3/BES0417210105
{'day_0': ['2021-01-05-15-44-20_video']}


### Imaging Preprocessing, Sptail filter, Motion correction, dF/F, PCA-ICA, Event detection
Note the following cell will perfrom the whole procedure from preprocessing to event detection, it's written in a huge for loop, so keep the identation before all codes.<br>
There will be outputs of file names for each completed steps, so you can check the working progress.

In [4]:
# Process each series all the way to event detection.
# output_dir = os.path.join(data_dir, 'processed_crop_pnr20_mincorr08')
# output_dir = os.path.join(data_dir, 'processed_crop_pnr10_mincorr08')
output_dir = os.path.join(data_dir, 'processed_crop_pnr10_mincorr08_v160_diam9')

# output_dir = os.path.join(data_dir, 'processed_crop_pnr15_mincorr08')

os.makedirs(output_dir, exist_ok=True)

proc_movie_files = []
proc_cs_files = []
for series_name, rec_names in series_rec_names.items():
    
    # Generate the recording file paths.
#     rec_files = [os.path.join(data_dir, name + '.xml') for name in rec_names]
    rec_files = [os.path.join(data_dir, name + '.isxd') for name in rec_names] ## This line is for new recording from nVista3
    print(rec_files) # for progress feedback, not necessary
    
    # Preprocess the recordings by spatially downsampling by a factor of 4.
    pp_files = isx.make_output_file_paths(rec_files, output_dir, 'PP')
    # for DSC012972
#     isx.preprocess(rec_files, pp_files, spatial_downsample_factor=1, crop_rect=[10,48,151,282])
    # for DSC013919
#     isx.preprocess(rec_files, pp_files, spatial_downsample_factor=1, crop_rect=[45,186,124,242])
    # for BES0417
    isx.preprocess(rec_files, pp_files, spatial_downsample_factor=1, crop_rect=[12,31,199,268])
    # for BES0424
#     isx.preprocess(rec_files, pp_files, spatial_downsample_factor=1, crop_rect=[15,14,171,236])
    # for BES0427
#     isx.preprocess(rec_files, pp_files, spatial_downsample_factor=1, crop_rect=[8,10,178,228])
    # for BES0225
#     isx.preprocess(rec_files, pp_files, spatial_downsample_factor=1, crop_rect=[10,18,199,273])


    print(pp_files)
    
    # Perform spatial bandpass filtering with defaults.
    bp_files = isx.make_output_file_paths(pp_files, output_dir, 'BP')
    isx.spatial_filter(pp_files, bp_files, low_cutoff=0.005, high_cutoff=0.500)
    print(bp_files)
    
    # Motion correct the movies using the mean projection as a reference frame.
    mean_proj_file = os.path.join(output_dir, '{}-mean_image.isxd'.format(series_name))
    isx.project_movie(bp_files, mean_proj_file, stat_type='mean')
    mc_files = isx.make_output_file_paths(bp_files, output_dir, 'MC')
    translation_files = isx.make_output_file_paths(mc_files, output_dir, 'translations', 'csv')
    crop_rect_file = os.path.join(output_dir, '{}-crop_rect.csv'.format(series_name))
    isx.motion_correct(
         bp_files, mc_files, max_translation=20, reference_file_name=mean_proj_file,
         low_bandpass_cutoff=None, high_bandpass_cutoff=None,
         output_translation_files=translation_files, output_crop_rect_file=crop_rect_file)
    print(mc_files)
    
    # Run DF/F on the motion corrected movies.
    dff_files = isx.make_output_file_paths(mc_files, output_dir, 'DFF')
    isx.dff(mc_files, dff_files, f0_type='mean')
    print(dff_files)
    
#     # Run PCA-ICA on the DF/F movies.
#     # Note that you will have to manually determine the number of cells, which we
#     # determined here as 180.
#     # Increase the block_size to increase speed at the expense of more memory usage.
#     ic_files = isx.make_output_file_paths(dff_files, output_dir, 'PCA_ICA')
#     num_cells = 180
#     isx.pca_ica(dff_files, ic_files, num_cells, int(1.15 * num_cells), block_size=1000)
#     #print(ic_files)

#     # Run event detection on the PCA-ICA cell sets.
#     event_files = isx.make_output_file_paths(ic_files, output_dir, 'ED')
#     isx.event_detection(ic_files, event_files, threshold=5)
#     print(event_files)

#     # Automatically accept and reject cells based on their cell metrics
#     # Only accept cells that have a nonzero event rate, an SNR greater
#     # than 3, and only one connected component after thresholding
#     auto_ar_filters = [('SNR', '>', 3), ('Event Rate', '>', 0), ('# Comps', '=', 1)]
#     isx.auto_accept_reject(ic_files, event_files, filters=auto_ar_filters)

#     # Store the processed movies and cell sets for longitudinal registration.
#     proc_movie_files += dff_files
#     proc_cs_files += ic_files
#     print(proc_movie_files)

['F:/inscopix_nVista3/BES0417210105\\2021-01-05-15-44-20_video.isxd']
['F:/inscopix_nVista3/BES0417210105\\processed_crop_pnr10_mincorr08_v160_diam9\\2021-01-05-15-44-20_video-PP.isxd']
['F:/inscopix_nVista3/BES0417210105\\processed_crop_pnr10_mincorr08_v160_diam9\\2021-01-05-15-44-20_video-PP-BP.isxd']
['F:/inscopix_nVista3/BES0417210105\\processed_crop_pnr10_mincorr08_v160_diam9\\2021-01-05-15-44-20_video-PP-BP-MC.isxd']
['F:/inscopix_nVista3/BES0417210105\\processed_crop_pnr10_mincorr08_v160_diam9\\2021-01-05-15-44-20_video-PP-BP-MC-DFF.isxd']


In [5]:
end_time = datetime.now()
print('Duration from starting to MC: {}'.format(end_time - start_time))

Duration from starting to MC: 0:07:26.659847


You should see some outputs from each step of precessing above here.


If you have the MC.isxd file already, you can continue with following code.

'F:/inscopix_new/DSC7674200225\\processed\\recording_20200225_153103-PP-BP-MC.isxd'

### ROI detection by CNMFe
Now, we finished all preprocessing from the inscopix, and can move forward for Ca<sup>2</sup> ROI detection by CNMFe. First we would need to define our parameters.

In [6]:
# To start with CNMFe, we can try to set up a cluster on our computer so 
# it can process much faster. Go to anaconda promt, activate isxenv, then run
# ipcluster start -n 16

# Or less amount
# Then use a different terminal to start the jupyter notebook.
# Check again if it's possible to have this at the beginning of the notebook

In [7]:
# ## Test with different setting
# # Define the motion correct filename for further process.
mov_file = str(mc_files) # This should have the full path to the motion corrected file.

# Create lists of CNMFe output files (cell traces and events)
# cellset_list = []
# cellset_list.append(os.path.join(output_dir, mov_file[-41:-7]+'-CNMFE_13'+ '.isxd'))
# cellset_list.append(os.path.join(output_dir, mov_file[-41:-7]+'-CNMFE_17'+ '.isxd'))
# cellset_list.append(os.path.join(output_dir, mov_file[-41:-7]+'-CNMFE_19'+ '.isxd'))
# cellset_list.append(os.path.join(output_dir, mov_file[-41:-7]+'-CNMFE_21'+ '.isxd'))
# for cellset in cellset_list:
#     print(cellset)

# events_list = []
# events_list.append(os.path.join(output_dir, mov_file[-41:-7]+'-CNMFE_13-ED'+ '.isxd'))
# events_list.append(os.path.join(output_dir, mov_file[-41:-7]+'-CNMFE_17-ED'+ '.isxd'))
# events_list.append(os.path.join(output_dir, mov_file[-41:-7]+'-CNMFE_19-ED'+ '.isxd'))
# events_list.append(os.path.join(output_dir, mov_file[-41:-7]+'-CNMFE_21-ED'+ '.isxd'))
# for events in events_list:
#     print(events)

In [8]:
cell_set = os.path.join(output_dir, mov_file[-41:-7]+'-CNMFE_new.isxd')
cell_set

cell_set_denoised = os.path.join(output_dir, mov_file[-41:-7]+'-CNMFE_new_denoised.isxd')
cell_set_denoised

cell_set_event = os.path.join(output_dir, mov_file[-41:-7]+'-CNMFE_new_event.isxd')
cell_set_event                                      

'F:/inscopix_nVista3/BES0417210105\\processed_crop_pnr10_mincorr08_v160_diam9\\2021-01-05-15-44-20_video-PP-BP-MC-CNMFE_new_event.isxd'

In [9]:
start_time_2 = datetime.now()
print('Start time 2: {}'.format(start_time_2))

Start time 2: 2021-08-11 14:59:54.623112


In [10]:
# # CNMFe parameter setting
# # Define cell size, with the Ca2+ imaging from the inscopix (w/o crop VoF), 
# # a 4x spatial downsampling video usually has a cell size around 15-20 pixel.
# # Belows are some parameters that I tried that can give us nice yield of cells (~200-300 cells)

# # Most important is the cell size here, change this one.
# cell_size_px = 19       # commonly use: 13, 17, 21

# px_set = [13,17,19,21]     # Use for automatic naming
# gSig=(cell_size_px-1)/4 # gSig
# gSiz=cell_size_px       # gSiz = gSig*4+1 (from CNMFe manual)
# # num_processes=1         # Check again if we can do multi-processor by changing the ipython kernel
# ## New parameters
# processing_mode='parallel_patches'
# num_threads=4
# patch_size=80
# patch_overlap=20
# output_unit_type=’df_over_noise’

# K=None
# rf=[40,40]
# stride=20
# min_pnr=10              # 10 for PV, 20 for CaMKII
# min_corr=0.8            # Try 0.7 corr
# event_threshold=0.3     # Used 0.1 before, too noisy

In [11]:
## CNMFe new parameters
input_movie_files      = mc_files
output_cell_set_files  = cell_set
cell_diameter          = 9 # Check again what is the good values for us, or ask Alexia for the details if this is actually the gSig and gSiz before?
min_corr               = 0.8
min_pnr                = 10
bg_spatial_subsampling = 2
ring_size_factor       = 1.4
gaussian_kernel_size   = 0
closing_kernel_size    = 0
merge_threshold        = 0.7
processing_mode        = 'parallel_patches'
num_threads            = 8 # Don't go higher than 8 threads
patch_size             = 80
patch_overlap          = 20
output_unit_type       = 'scaled_df' # df_over_noise

## For OASIS deconvolution
input_raw_cellset_files       = cell_set
output_denoised_cellset_files = cell_set_denoised
output_spike_eventset_files   = cell_set_event
accepted_only                 = False
spike_snr_threshold           = 3.75 # this is close to what we had before
noise_range                   = (0.25, 0.5)
noise_method                  = 'mean'
first_order_ar                = True
lags                          = 5
fudge_factor                  = 0.96
deconvolution_method          = 'oasis'

In [ ]:
isx.run_cnmfe(input_movie_files = input_movie_files,
              output_cell_set_files = output_cell_set_files,
              output_dir = output_dir, 
              cell_diameter = cell_diameter,
              min_corr = min_corr, min_pnr = min_pnr,
              bg_spatial_subsampling = bg_spatial_subsampling,
              ring_size_factor = ring_size_factor, 
              gaussian_kernel_size = gaussian_kernel_size,
              closing_kernel_size = closing_kernel_size,
              merge_threshold = merge_threshold,
              processing_mode = processing_mode,
              num_threads = num_threads, patch_size = patch_size,
              patch_overlap = patch_overlap, 
              output_unit_type = output_unit_type
              )
print('Finished CNMFe '+output_cell_set_files)

In [ ]:
isx.deconvolve_cellset(input_raw_cellset_files = input_raw_cellset_files,
                       output_denoised_cellset_files = output_denoised_cellset_files,
                       output_spike_eventset_files= output_spike_eventset_files,
                       accepted_only = accepted_only,
                       spike_snr_threshold = spike_snr_threshold,
                       noise_range = noise_range, 
                       noise_method = noise_method,
                       first_order_ar = first_order_ar,
                       lags = lags, fudge_factor = fudge_factor,
                       deconvolution_method = deconvolution_method
                       )
print('Finished deconvolution '+output_spike_eventset_files)

In [ ]:
## Remove unnecessary files to sace storage space
os.remove(pp_files[0])
print('Removed '+str(pp_files[0]))
os.remove(bp_files[0])
print('Removed '+str(bp_files[0]))

os.chdir(output_dir) # set directory
for file in glob.glob("*.tiff"):
    os.remove(file)
    print('Remove '+str(file))
    
# for file in glob.glob("*.mmap"):
#     os.remove(file)
#     print('Remove '+str(file))

In [ ]:
end_time = datetime.now()
print('End time: {}'.format(end_time))
print('Duration for CNMFe: {}'.format(end_time - start_time_2))
print('Duration for whole process: {}'.format(end_time - start_time))